In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts
import hvplot.pandas
import panel as pn
from bokeh.io import output_notebook

output_notebook()
hv.extension('bokeh')
pn.extension('bokeh')

font_dict = {'title': 16, 'labels': 14, 'ticks': 12, 'legend': 12}
hv.opts.defaults(
    hv.opts.Curve(width=600, height=400, tools=['hover'], fontsize=font_dict),
    hv.opts.Scatter(width=600, height=400, size=8, tools=['hover'], fontsize=font_dict),
    hv.opts.Histogram(width=600, height=400, fontsize=font_dict),
    hv.opts.Bars(width=600, height=400, fontsize=font_dict),
)


Loading BokehJS ...

In [16]:
monkey = 'Test'
rows = []
for i in range(1, 5):
    rows += [
        {
            'type': 'STOP',
            'trial_failed': True,
            'computed_rt': i*20 + n, 
            'ssd_number': i,    
        }
        for n in range(0, 40)
    ] + [
        {
            'type': 'CONT',
            'trial_failed': False,
            'computed_rt': i*100 + n, 
            'ssd_number': i,    
        } 
        for n in range(0, 40)
    ] + [
        {
            'type': 'STOP',
            'trial_failed': False,
            'computed_rt': i*20 + n, 
            'ssd_number': i,    
        }
        for n in range(0, 60)
    ] + [
        {
            'type': 'CONT',
            'trial_failed': True,
            'computed_rt': i*100 + n, 
            'ssd_number': i,    
        } 
        for n in range(0, 60)
]
df_rt = pd.DataFrame(rows)
df_rt

,type,trial_failed,computed_rt,ssd_number
0,STOP,True,20,1
1,STOP,True,21,1
2,STOP,True,22,1
3,STOP,True,23,1
4,STOP,True,24,1
...,...,...,...,...
795,CONT,True,455,4
796,CONT,True,456,4
797,CONT,True,457,4
798,CONT,True,458,4


In [18]:
failed_stop_trials = df_rt[df_rt['type'].isin(['STOP']) & df_rt['trial_failed'].isin([True])]
successful_cont_trials = df_rt[df_rt['type'].isin(['CONT']) & df_rt['trial_failed'].isin([False])]

tot_cont_trials = len(df_rt[df_rt['type'].isin(['CONT'])])
tot_stop_trials = len(df_rt[df_rt['type'].isin(['STOP'])])
tot_trial = tot_cont_trials + tot_stop_trials

print(f"Total CONT trials: {tot_cont_trials:,}")
print(f" Total STOP trials: {tot_stop_trials:,}")

def frame_it(df, normalizer, ssd_prefix):
    tmp_df = df.groupby(['computed_rt', 'ssd_number']).size()
    # print(tmp_df)
    tmp_df = tmp_df.reset_index().rename(columns={'computed_rt': 'Reaction Time', 'ssd_number': 'SSD Number', 0: 'Count'})
    tmp_df['percentage'] = (tmp_df['Count'] / normalizer) * 100
    # tmp_df['percentage'] = (tmp_df['Count'] / 1) #* 100
    tmp_df['SSD Number'] = tmp_df.apply(lambda row: f'{ssd_prefix}{int(row["SSD Number"])}', axis=1)
    tmp_df = tmp_df.groupby(['Reaction Time', 'SSD Number'])['percentage'].sum().reset_index()
    # Group 'Reaction Time' into 20 millisecond bins and sum percentages within each bin
    tmp_df['Reaction Time Bin'] = (tmp_df['Reaction Time'] // 20) * 20
    tmp_df = tmp_df.groupby(['Reaction Time Bin', 'SSD Number'], as_index=False)['percentage'].sum()
    return tmp_df

cont_df = frame_it(successful_cont_trials, tot_cont_trials, 'CSD')
stop_df = frame_it(failed_stop_trials, tot_stop_trials, 'SSD')

cont_plot = cont_df.hvplot.line(
    x='Reaction Time Bin', y='percentage', by='SSD Number',
    title=f'{monkey.title()} - Continue and error stop RT',
    xlabel='Reaction time (ms)',
    ylabel='Percentage of total trials',
    width=800, height=400,
    line_dash='dashed',
    line_width=3,
)

stop_plot = stop_df.hvplot.line(
    x='Reaction Time Bin', y='percentage', by='SSD Number',
    line_width=3, 
    xlim=(0, 600)
)

(cont_plot * stop_plot).opts(legend_position='top_right')

# stop_df
# cont_df

Total CONT trials: 400
 Total STOP trials: 400


:Overlay
   .NdOverlay.I  :NdOverlay   [SSD Number]
      :Curve   [Reaction Time Bin]   (percentage)
   .NdOverlay.II :NdOverlay   [SSD Number]
      :Curve   [Reaction Time Bin]   (percentage)